In [ ]:
!pip install openweathermapy
!pip install geopip
!pip install numpy 
!pip install cython
!pip install --upgrade geohash-hilbert

In [1]:
import openweathermapy.core as owm
import requests
import geopip
import os
import datetime as dt

In [2]:
# Weather API
from config import ow_api_key

# MapQuest API
KEY = '6aMCYSpGBo4Eg20Lw7RljQ0nXcUGsA5S'

# Set geopip to look at geojson file
os.environ['REVERSE_GEOCODE_DATA'] = 'Resources/Data/community_areas.geojson'

In [3]:
# Form Inputs

start_address = '405 N Wabash Avenue, Chicago, IL'
end_address = '938 W Webster Ave, Chicago, IL 60614'
time = '2019-12-11 00:00:00'
# make default app time the current time

In [4]:
# format time to default

time = '2019-12-11 00:00:00'
noncoded_time = dt.datetime.today()
# if error then use 'dt.datetime.today()'

In [5]:
def coded_week(date_input):
    weekdays = 7     

    coded_week = []
    for i in range(weekdays):
        if i == date_input-1:
            coded_week.append(1)
        else:
            coded_week.append(0)
        
    return coded_week

In [6]:
# List of Coded Weekday Values
start_date = dt.datetime.today().weekday()
coded_week = coded_week(start_date)

In [7]:
def coded_date(list, type):
    if type == 'months':
        time_frame = 12
        list_var = 0
        
    elif type == 'days':
        if list[0] in [1,3,5,7,8,10,12]:
            time_frame = 31
            list_var= 1
        
        if list[0] in [4,6,9,11]:
            time_frame = 30
            list_var= 1
        
        if list[0] in [2]:
            time_frame = 28
            list_var= 1
        
        
    elif type == 'hr-mins':
        time_frame = 96
        hours = 24
        hour = list[2]
        minute = list[3]
        quarter_hour = []
        

        for j in range(hours):
            if j == hour:
                if minute >=45:
                    quarter_hour += [0,0,0,1]
                elif minute >=30:
                    quarter_hour += [0,0,1,0]
                elif minute >=15:
                    quarter_hour += [0,1,0,0]
                else:
                    quarter_hour += [1,0,0,0]
            else:
                quarter_hour += [0,0,0,0]

     
    if type in ['months', 'days']:
        coded_time = []
        for i in range(time_frame):
            if i == list[list_var]-1:
                coded_time.append(1)
            else:
                coded_time.append(0)
        return coded_time
    else:
        return quarter_hour

In [8]:
# List of Coded Months and Days (96 Quarter Hour Periods)
noncoded_month = noncoded_time.month
noncoded_day = noncoded_time.day
noncoded_hour = noncoded_time.hour
noncoded_minute = noncoded_time.minute
date_list = [noncoded_month, noncoded_day, noncoded_hour, noncoded_minute]
coded_dates = coded_date(date_list, 'months')+coded_date(date_list, 'days')+coded_date(date_list, 'hr-mins')

In [9]:
months = coded_date(date_list, 'months')
months[10]

0

In [10]:
day_times = coded_date(date_list, 'hr-mins')

In [11]:
# Extract Map Information - variables added to model
url = f'https://www.mapquestapi.com/directions/v2/optimizedRoute?json={{"locations":["{start_address}","{end_address}"]}}&outFormat=json&key={KEY}'
format = "json"

street_directions = requests.get(f"{url}").json()
distance = street_directions["route"]['distance']
duration = street_directions["route"]["time"]
start_longitude = street_directions["route"]["boundingBox"]["lr"]['lng']
start_latitude = street_directions["route"]["boundingBox"]["lr"]['lat']
end_longitude = street_directions["route"]["boundingBox"]["ul"]['lng']
end_latitude = street_directions["route"]["boundingBox"]["ul"]['lat']


In [12]:
def coded_community(community):
    number_communities = 77     

    coded_location = []
    for i in range(number_communities):
        if i == community-1:
            coded_location.append(1)
        else:
            coded_location.append(0)
        
    return coded_location

In [13]:
# List of Coded Start Community
start_geo = int(geopip.search(start_longitude, start_latitude)['area_num_1'])
coded_start_community = coded_community(start_geo)

In [14]:
coded_start_community[75]

0

In [15]:
# List of Coded End Community
end_geo = int(geopip.search(end_longitude, end_latitude)['area_num_1'])
coded_end_community = coded_community(end_geo)

In [24]:
# Retrieve Weather Data
settings = {"units": "imperial", "appid": ow_api_key}
location = (start_latitude, start_longitude)
current_weather_chicago = owm.get_current(location, **settings)
actual_temp = current_weather_chicago('main.temp')
feels_temp = current_weather_chicago('main.feels_like')
pressure = current_weather_chicago('main.pressure')
humidity = current_weather_chicago('main.humidity')

# Precipitation calc
# https://openweathermap.org/weather-conditions
# https://openweathermap.org/weather-data

# weather code converted into average precipitation value
rain_light_precipitation = [200, 230, 231, 300, 301, 310, 311, 500, 520]
rain_moderate_precipitation = [201, 232, 302, 312, 313, 321, 501, 521]
rain_intense_precipitation = [202, 314, 502, 503, 504, 531]

weather_code = current_weather_chicago('cod')

if weather_code in rain_light_precipitation:
    rain = 0.1
elif weather_code in rain_moderate_precipitation:
    rain = 1.0
elif weather_code in rain_intense_precipitation:
    rain = 2.0
else:
    rain = 0.0

snow_light_precipitation = [600, 620]
snow_moderate_precipitation = [601, 612, 613, 615, 616, 621]
snow_intense_precipitation = [602, 611, 622]

if weather_code in snow_light_precipitation:
    snow = 0.1
elif weather_code in snow_moderate_precipitation:
    snow = 1.0
elif weather_code in snow_intense_precipitation:
    snow = 2.0
else:
    snow = 0.0

weather = [actual_temp, feels_temp, pressure, humidity, rain, snow]

In [ ]:
# for rain it might not show up in the request becasue there is not key-value pair is there is no rain = need to create /
# logic that adds 0

In [25]:
weather

[3.51, -8.91, 1030, 55, 0.1, 0.0]

In [18]:
location

(41.889717, -87.626656)

In [26]:
# ML_input2 = [duration, distance, start_geo, end_geo, start_longitude, start_latitude, end_longitude, end_latitude] + coded_week + coded_date + coded_start_community + coded_end_community + weather
ML_input = [duration, distance, rain, coded_start_community[75], coded_end_community[75], actual_temp, coded_dates[10], coded_start_community[55], coded_end_community[55], snow]

In [27]:
ML_input

[476, 3.266, 0.1, 0, 0, 3.51, 0, 0, 0, 0.0]

In [23]:
len(ML_input)

10

In [21]:
import pandas as pd


In [22]:
pd.DataFrame(ML_input)

,0
0,476.000
1,3.266
2,0.100
3,0.000
4,0.000
5,3.700
6,0.000
7,0.000
8,0.000
9,0.100
